!pip install census

!pip install us

In [1]:
from census import Census
from us import states
import pandas as pd
import numpy as np
import pylab as pl
import csv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### Insert your census api key her below

In [2]:
c = Census("4c26aa6ebbaef54a55d3903212eabbb506ade381")

#### This is the data dictionary

#### 2000 SF3 Variables

#### Creating the buckets for variables that are separeted into bins like income

In [3]:
buckets = [0, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 60000, 75000, 100000, 125000, 150000, 200000,1000000]

#### Insert the variables you want to download. The second part is just so it is automated.

In [4]:
variables=['P0060', 'H0620', 'H0630', 'H0850', 'H0070', 'P0520', 'P0530', 'P0370']

In [5]:
var=[]
for i in variables:
    var.append(i+'01')
    if ((i == 'P0060') | (i == 'H0620') | (i == 'H0070')):
        var.append(i+'02')
    if i == 'H0070':
        var.append(i+'03')

In [6]:
var.append('NAME')
for i in range(2,10):
    var.append('P05200'+str(i))
for i in range(10,18):
    var.append('P0520'+str(i))

In [7]:
var.append('P037015')
var.append('P037032')

#### Choose the state and insert its code (as a string number)

In [8]:
state = '09'

In [9]:
sql_query='state:{} county:*'.format(state)

In [10]:
dic = c.sf3.get(var, geo = {'for': 'tract:*',
                       'in': sql_query}, year = 2000)

In [11]:
df = pd.DataFrame.from_dict(dic)

In [12]:
df = df.reset_index()

In [13]:
df = df.drop(columns = 'index')

#### Exporting the raw file

In [14]:
df.to_csv('variables_' + state + '.csv', index = None)

#### Calculating the thresholds for each of the counties

In [15]:
county = df.county.unique()

In [16]:
counties = {}
counties = counties.fromkeys(county)

In [17]:
columns = df.columns

In [18]:
for i in range(0,len(columns)-3):
    if i!=7:
        df[columns[i]] = df[columns[i]].astype(float)

In [19]:
for i in county:
    counties[i] = np.median(df[df.county == i].P053001)

In [20]:
df_threshold = pd.DataFrame.from_dict(counties, orient='index')

In [21]:
df_threshold = df_threshold.rename(columns={0:'median'})

In [22]:
#df_typo['vvli_threshold']=0.3*df_typo['median']
df_threshold['vli_threshold'] = 0.5*df_threshold['median']
df_threshold['li_threshold'] = 0.8*df_threshold['median']
df_threshold['mi_threshold'] = 1.2*df_threshold['median']
df_threshold['mhi_threshold'] = 1.5*df_threshold['median']
df_threshold['hi_threshold'] = 2*df_threshold['median']
df_threshold['vhi_threshold'] = 1000000

#### Creating the different typologies based on each county's median income

In [23]:
table = np.zeros((len(county),len(buckets)-1))
for j in range(0, len(counties)):
    col = 1
    for i in range(0, len(buckets)-1):
        if buckets[i+1] < df_threshold.iloc[j, col]:
            table[j, i] = True
        else:
            table[j, i] = (df_threshold.iloc[j, col] - buckets[i]) / (buckets[i+1] - buckets[i])
            col = col+1

In [24]:
values=[]
for i in county:
    with open('variables_'+state+'.csv', 'r') as fi:
        reader = csv.DictReader(fi)
        def creator(reader, c):
            bt = filter(lambda x: x['county'] == c, reader)
            return (list(bt))
        value = creator(reader, i)
    values.append(value)

In [25]:
income_typo = []
for j in range(0, len(county)):
    for i in range(0, len(values[j])):
        typo = []
        summ = 0
        for col in range(0,len(buckets)-1):
            if ((table[j, col] == 1) & (values[j][i][var[9]]!='0')):
                summ = summ + float(values[j][i][var[13+col]])/float(values[j][i][var[9]])
            else:
                if values[j][i][var[9]]!='0':
                    summ = summ + float(values[j][i][var[13+col]])*table[j, col]/float(values[j][i][var[9]])
                    typo.append(summ)
                    summ = float(values[j][i][var[13+col]])*(1-table[j, col])/float(values[j][i][var[9]])
        typo.append(summ)
        income_typo.append(typo)

In [26]:
df_final = pd.DataFrame(income_typo)

In [27]:
df_final = df_final.rename(columns={0:'vli_00', 1:'li_00', 2:'mi_00', 3:'mhi_00', 4:'hi_00', 5:'vhi_00'})

In [28]:
df['col_00'] = df['P037015'] + df['P037032']

In [29]:
moe_names = ['per_col_00', 'per_rent_00', 'per_owners_00', 'per_nonwhite_00']
moe_dividend = ['col_00', 'H007003', 'H007002', 'P006002']
moe_divisor = ['P037001', 'H007001', 'H007001', 'P006001']

In [30]:
for i in range(0,len(moe_names)):
    if i!=3:
        df_final[moe_names[i]] = df[moe_dividend[i]]/df[moe_divisor[i]]
    else:
        df_final[moe_names[i]] = 1 - df[moe_dividend[i]]/df[moe_divisor[i]]

In [31]:
df = df.rename(columns={'P052001':'hh_00', 'H063001':'mrent_00', 'P053001':'hinc_00', 
                        'H085001':'mhval_00', 'H007001':'hu_00', 'P006001':'pop_00'})

In [32]:
df_final = df_final.join(df[['hh_00', 'mrent_00', 'hinc_00', 'mhval_00', 'hu_00', 'pop_00', 'NAME', 'state', 'county',
                             'tract']])

#### Droping null values

In [33]:
prev_len = len(df_final)

In [34]:
df_final = df_final.dropna(axis = 0, how = 'any')

In [35]:
final_len = len(df_final)

In [43]:
print('The percentage of information lost is: {}'.format((prev_len-final_len)*100/prev_len))

The percentage of information lost is: 0.8547008547008547


#### Exporting the final file

In [37]:
df_final.to_csv(state+'_2000_variables.csv', index = None)